In [34]:
import fmpy
import logging

_logger = logging.getLogger(__name__)

class FmuLoader:
    def __init__(self, fmu_file) -> None:
        
        self.model_description = fmpy.read_model_description(fmu_file)  
        self.fmu_name = self.model_description.modelName
        self.unzipdir = fmpy.extract(fmu_file)
        self.fmu = fmpy.fmi2.FMU2Slave(guid=self.model_description.guid,
                        unzipDirectory=self.unzipdir,
                        modelIdentifier=self.model_description.coSimulation.modelIdentifier,
                        instanceName='instance1')
        self.fmu.instantiate()
        self.fmu.enterInitializationMode()      
        self.fmu.exitInitializationMode()
        self.fmu_inputs = {}
        self.fmu_outputs = {}
        self.fmu_parameters = {}
        self.locate_variable_names()

    def _add_input(self, variable:fmpy.model_description.ScalarVariable):
        self.fmu_inputs[variable.name] = {
            "id" : variable.valueReference,
            "type" : variable.type
        }
        return

    def _add_output(self, variable:fmpy.model_description.ScalarVariable):
        self.fmu_outputs[variable.name] = {
            "id" : variable.valueReference,
            "type" : variable.type
        }
        
    def locate_variable_names(self):

        # Setting up
        _logger.info("Run simulation \n")            

        # Gathering the I/Os references TODO: Outside of simulation loop
        for variable in self.model_description.modelVariables:
            if variable.causality == "input":
                self._add_input(variable=variable)
            elif variable.causality == "output":
                self._add_output(variable=variable)
                
        print(f"inp = {self.fmu_inputs}, \nout = {self.fmu_outputs}, \npar = {self.fmu_parameters}")

    def get_fmu_inputs(self):
        return list(self.fmu_inputs.keys())
    
    def get_fmu_outputs(self):
        return list(self.fmu_outputs.keys())

# simulation loop 1
fmu = FmuLoader(fmu_file="../FMUs/LOC_CNTRL_custom_linux.fmu") 
fmu.fmu.setReal([fmu.fmu_inputs["SETPOINT_temperature_lube_oil"]["id"]], [0])  
fmu.fmu.setReal([fmu.fmu_inputs["INPUT_temperature_lube_oil"]["id"]], [10])  
fmu.fmu.doStep(currentCommunicationPoint=0, communicationStepSize=5.0) 
fmu.fmu.getReal([fmu.fmu_outputs["OUTPUT_control_valve_position"]["id"]])


inp = {'SETPOINT_temperature_lube_oil': {'id': 0, 'type': 'Real'}, 'INPUT_temperature_lube_oil': {'id': 1, 'type': 'Real'}}, 
out = {'OUTPUT_control_valve_position': {'id': 2, 'type': 'Real'}}, 
par = {}


[0.7333333333333333]

In [37]:
fmu.fmu_name

'LOC_CNTRL_v2_customPI'